In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
from wordcloud import STOPWORDS

## Import dataset

In [2]:
train = pd.read_csv('../dataset/train.csv')
test = pd.read_csv('../dataset/test.csv')

## Preprocessing

In [3]:
test_id = test.id
train_id = train.id

#drop id and location
c = ['id', 'location']
train = train.drop(columns=c)
test = test.drop(columns=c)

#### Missing Value

In [4]:
#fill missing keywords with unknown
train['keyword'] = train.keyword.fillna('unknown')
test['keyword'] = test.keyword.fillna('unknown')

#fill keyword to tweets
train['text'] = train['text'] + ' ' + train['keyword']
test['text'] = test['text'] + ' ' + test['keyword']

#### Truncations 

In [6]:
df_truncation = pd.read_csv('../dataset/Truncations.csv').set_index('index')
df_truncation.head()

,Truncation
index,
ain't,am not / are not / is not / has not / have not
aren't,are not / am not
can't,cannot
can't've,cannot have
'cause,because


In [7]:
truncation = df_truncation.Truncation.to_dict()
truncation

{"ain't": 'am not / are not / is not / has not / have not',
 "aren't": 'are not / am not',
 "can't": 'cannot',
 "can't've": 'cannot have',
 "'cause": 'because',
 "could've": 'could have',
 "couldn't": 'could not',
 "couldn't've": 'could not have',
 "didn't": 'did not',
 "doesn't": 'does not',
 "don't": 'do not',
 "hadn't": 'had not',
 "hadn't've": 'had not have',
 "hasn't": 'has not',
 "haven't": 'have not',
 "he'd": 'he had / he would',
 "he'd've": 'he would have',
 "he'll": 'he shall / he will',
 "he'll've": 'he shall have / he will have',
 "he's": 'he has / he is',
 "how'd": 'how did',
 "how'd'y": 'how do you',
 "how'll": 'how will',
 "how's": 'how has / how is / how does',
 "I'd": 'I had / I would',
 "I'd've": 'I would have',
 "I'll": 'I shall / I will',
 "I'll've": 'I shall have / I will have',
 "I'm": 'I am',
 "I've": 'I have',
 "isn't": 'is not',
 "it'd": 'it had / it would',
 "it'd've": 'it would have',
 "it'll": 'it shall / it will',
 "it'll've": 'it shall have / it will have'

In [20]:
#compile all contraction words
truncations_re = re.compile('(%s)' % '|'.join(truncation.keys()))


#define function to expand contractions and showcase
def expand_contractions(s, contractions = truncation):
#     print(s)
    def replace(match):
#         print( contractions[match.group(0)])
        return contractions[match.group(0)]
    return truncations_re.sub(replace, s)

expand_contractions("ain't")

ain't


'am not / are not / is not / has not / have not'

#### Strip Sentence


In [21]:
def strip(x):
    return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",x).split())

strip('@shawn Titanic #tragedy could have been prevented Economic Times: Telegraph.co.uk Titanic tragedy could have been prevented... http://bet.ly/tuN2wx')

'Titanic tragedy could have been prevented Economic Times Telegraph co uk Titanic tragedy could have been prevented'